# NLP

## Sentiment analysis

## Identifacação do sentimento (positivo, negativo) em reviews do IMDB

## import padrão

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('../datasets/IMDB Dataset.csv')

## EDA

In [3]:
df.shape
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

## ==> Dataset balanceado

In [5]:
df.dtypes

review       object
sentiment    object
dtype: object

## ==> Ambas features do tipo object

In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

## ==> Não há dados faltando

In [7]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


## ==> Quase todas as reviews são diferentes
## ==> Apenas 2 sentimentos (positivo, negativo)
## ==> Necessário remover pontuação e marcações

## Pre-processamento
- Remoção de marcações HTML
- Remoção de pontuação e caracteres especiais
- Remoção de stopwords
- Stemização / lematização para obter o tronco comum das palavras
- vetorização para codificar dados textuais

In [8]:
# Vamos usar um exemplo
exemplo = df.iloc[1].review
exemplo

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

## pip install bs4

In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(exemplo, 'html.parser')
exemplo = soup.get_text()
exemplo

'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well done.'

In [10]:
# Remover tudo excepto Letras usando expressões regulares
import re
exemplo = re.sub('<.*?>', '',exemplo)
exemplo = re.sub(r'\[[^]]*','',exemplo)
exemplo = re.sub(r'\d+','',exemplo)
exemplo = re.sub('[^a-zA-Z]',' ',exemplo)
exemplo

'A wonderful little production  The filming technique is very unassuming  very old time BBC fashion and gives a comforting  and sometimes discomforting  sense of realism to the entire piece  The actors are extremely well chosen  Michael Sheen not only  has got all the polari  but he has all the voices down pat too  You can truly see the seamless editing guided by the references to Williams  diary entries  not only is it well worth the watching but it is a terrificly written and performed piece  A masterful production about one of the great master s of comedy and his life  The realism really comes home with the little things  the fantasy of the guard which  rather than use the traditional  dream  techniques remains solid then disappears  It plays on our knowledge and our senses  particularly with the scenes concerning Orton and Halliwell and the sets  particularly of their flat with Halliwell s murals decorating every surface  are terribly well done '

In [11]:
# pip install nltk

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [14]:
exemplo = exemplo.split()
exemplo = [word for word in exemplo if not word in set(stopwords.words('english'))]
exemplo

['A',
 'wonderful',
 'little',
 'production',
 'The',
 'filming',
 'technique',
 'unassuming',
 'old',
 'time',
 'BBC',
 'fashion',
 'gives',
 'comforting',
 'sometimes',
 'discomforting',
 'sense',
 'realism',
 'entire',
 'piece',
 'The',
 'actors',
 'extremely',
 'well',
 'chosen',
 'Michael',
 'Sheen',
 'got',
 'polari',
 'voices',
 'pat',
 'You',
 'truly',
 'see',
 'seamless',
 'editing',
 'guided',
 'references',
 'Williams',
 'diary',
 'entries',
 'well',
 'worth',
 'watching',
 'terrificly',
 'written',
 'performed',
 'piece',
 'A',
 'masterful',
 'production',
 'one',
 'great',
 'master',
 'comedy',
 'life',
 'The',
 'realism',
 'really',
 'comes',
 'home',
 'little',
 'things',
 'fantasy',
 'guard',
 'rather',
 'use',
 'traditional',
 'dream',
 'techniques',
 'remains',
 'solid',
 'disappears',
 'It',
 'plays',
 'knowledge',
 'senses',
 'particularly',
 'scenes',
 'concerning',
 'Orton',
 'Halliwell',
 'sets',
 'particularly',
 'flat',
 'Halliwell',
 'murals',
 'decorating',
 

### Stemming / Lematization
### Vamos aplicar e observar as diferenças em cada técnica

In [15]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [16]:
exemplo_stemmer = [stemmer.stem(word) for word in exemplo]
exemplo_stemmer = ' '.join(exemplo_stemmer)
exemplo_stemmer

'a wonder littl product the film techniqu unassum old time bbc fashion give comfort sometim discomfort sens realism entir piec the actor extrem well chosen michael sheen got polari voic pat you truli see seamless edit guid refer william diari entri well worth watch terrificli written perform piec a master product one great master comedi life the realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear it play knowledg sens particularli scene concern orton halliwel set particularli flat halliwel mural decor everi surfac terribl well done'

In [17]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [18]:
exemplo_lemmatizer = [lemmatizer.lemmatize(word) for word in exemplo]
exemplo_lemmatizer = ' '.join(exemplo_lemmatizer)
exemplo_lemmatizer

'A wonderful little production The filming technique unassuming old time BBC fashion give comforting sometimes discomforting sense realism entire piece The actor extremely well chosen Michael Sheen got polari voice pat You truly see seamless editing guided reference Williams diary entry well worth watching terrificly written performed piece A masterful production one great master comedy life The realism really come home little thing fantasy guard rather use traditional dream technique remains solid disappears It play knowledge sens particularly scene concerning Orton Halliwell set particularly flat Halliwell mural decorating every surface terribly well done'